In [2]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

In [3]:
Churn_Modelling = pd.read_csv('..\data\Churn_Modelling.csv')
Churn_Modelling

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
Churn_Modelling["Geography"].unique()
Churn_Modelling["Gender"].unique()

category = {
    "Geography": list(Churn_Modelling["Geography"].unique()),
    "Gender": list(Churn_Modelling["Gender"].unique())
}
category

{'Geography': ['France', 'Spain', 'Germany'], 'Gender': ['Female', 'Male']}

In [5]:
Churn_Modelling["Geography_cat"] = pd.Categorical(Churn_Modelling["Geography"], categories = category["Geography"], ordered = False)
Churn_Modelling["Geography_cat"] = Churn_Modelling["Geography_cat"].cat.codes
Churn_Modelling["Gender_cat"] = pd.Categorical(Churn_Modelling["Gender"], categories = category["Gender"], ordered = False)
Churn_Modelling["Gender_cat"] = Churn_Modelling["Gender_cat"].cat.codes
target = Churn_Modelling["Exited"]
dataset = Churn_Modelling.iloc[:,[14,15,6,7,8,9,10,11,12]]
dataset

,Geography_cat,Gender_cat,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0,0,42,2,0.00,1,1,1,101348.88
1,1,0,41,1,83807.86,1,0,1,112542.58
2,0,0,42,8,159660.80,3,1,0,113931.57
3,0,0,39,1,0.00,2,0,0,93826.63
4,1,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...
9995,0,1,39,5,0.00,2,1,0,96270.64
9996,0,1,35,10,57369.61,1,1,1,101699.77
9997,0,0,36,7,0.00,1,0,1,42085.58
9998,2,1,42,3,75075.31,2,1,0,92888.52


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Geography_cat    10000 non-null  int8   
 1   Gender_cat       10000 non-null  int8   
 2   Age              10000 non-null  int64  
 3   Tenure           10000 non-null  int64  
 4   Balance          10000 non-null  float64
 5   NumOfProducts    10000 non-null  int64  
 6   HasCrCard        10000 non-null  int64  
 7   IsActiveMember   10000 non-null  int64  
 8   EstimatedSalary  10000 non-null  float64
dtypes: float64(2), int64(5), int8(2)
memory usage: 566.5 KB


In [43]:
# Split
(trainX, testX, trainY, testY) = train_test_split(dataset, target, shuffle=True, test_size=0.75, random_state=1234)

In [44]:
# use DMatrix for xgbosot
dtrain = xgb.DMatrix(trainX, label=trainY)
dtest = xgb.DMatrix(testX, label=testY)

In [45]:
# set xgboost params
param = {
    'max_depth': 5,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 2}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations


In [46]:
#------------- numpy array ------------------
# training and testing - numpy matrices
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)

# extracting most confident predictions
best_preds = np.asarray([np.argmax(line) for line in preds])
print("Numpy array precision:", precision_score(testY, best_preds, average='macro'))




Numpy array precision: 0.8049577434566344


In [51]:
model1 = xgb.XGBClassifier()
train_model1 = model1.fit(trainX, trainY)
pred1 = train_model1.predict(testX)
print('Model 1 XGboost Report')
print(classification_report(testY, pred1))
print("\nAccuracy for XGBoost Model: %.2f%%" % (precision_score(testY, pred1) * 100))

Model 1 XGboost Report
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      5961
           1       0.76      0.45      0.56      1539

    accuracy                           0.86      7500
   macro avg       0.82      0.70      0.74      7500
weighted avg       0.85      0.86      0.84      7500


Accuracy for XGBoost Model: 75.97%


In [37]:
from sklearn.model_selection import GridSearchCV

param_test2b = {
 'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model5 = gsearch2b.fit(trainX, trainY)
pred5 = train_model5.predict(testX)
print('Model 5 XGboost Report')
print(classification_report(testY, pred5))
print("\nAccuracy for XGBoost Model: %.2f%%" % (precision_score(testY, pred5) * 100))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Model 5 XGboost Report
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      2009
           1       0.73      0.47      0.57       491

    accuracy                           0.86      2500
   macro avg       0.80      0.71      0.74      2500
weighted avg       0.85      0.86      0.85      2500


Accuracy for XGBoost Model: 72.70%


In [ ]:
# import dill as pickle

# with open('../models/model_rf.pk', 'wb') as file:
#     pickle.dump(model, file)

# with open('../models/category.pk', 'wb') as file:
#     pickle.dump(category, file)